In [3]:
#  Collect Inspection Data

import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [4]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [5]:
df_ppd.head(5)

,{7011B109-CFCA-8ED6-E053-6B04A8C075C1},280000,2018-06-04 00:00,IP4 5ES,S,N,F,3,Unnamed: 8,RANDWELL CLOSE,Unnamed: 10,IPSWICH,IPSWICH.1,SUFFOLK,A,A.1
0,{7011B109-CFCB-8ED6-E053-6B04A8C075C1},280000,2018-05-29 00:00,IP1 4BS,T,N,F,261,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
1,{7011B109-CFCC-8ED6-E053-6B04A8C075C1},170000,2018-04-27 00:00,IP4 4BH,T,N,F,31,NaN,PARADE ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
2,{7011B109-CFCD-8ED6-E053-6B04A8C075C1},246000,2018-05-25 00:00,IP1 6NB,S,N,F,42,NaN,ELMCROFT ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
3,{7011B109-CFCE-8ED6-E053-6B04A8C075C1},180000,2018-06-08 00:00,IP3 9LZ,T,N,F,48,NaN,WYNTERTON CLOSE,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A
4,{7011B109-CFCF-8ED6-E053-6B04A8C075C1},245000,2018-05-11 00:00,IP1 4BU,T,N,F,235,NaN,NORWICH ROAD,NaN,IPSWICH,IPSWICH,SUFFOLK,A,A


In [6]:
df_ppd.shape

(1030277, 16)

In [7]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [8]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [9]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [10]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [11]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [12]:
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
422,APPLEGARTH ROAD,2.400000e+06
856,BARONSMEAD ROAD,2.375000e+06
982,BEAUCLERC ROAD,2.480000e+06
1103,BELVEDERE DRIVE,2.340000e+06
1216,BICKENHALL STREET,2.208500e+06
1254,BIRCHLANDS AVENUE,2.217000e+06
1554,BRAMPTON GROVE,2.456875e+06


In [13]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac

In [14]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from geopy.distance import vincenty

In [15]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 856
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 856, dtype: object
item.Street only: BARONSMEAD ROAD
index: 982
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 982, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1103
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1103, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1216
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [16]:
geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [17]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2.435000e+06,"(29.805547, -98.475534)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
856,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
982,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)"
1103,BELVEDERE DRIVE,2.340000e+06,"(44.7075622, -63.5455988)"
1216,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)"
1254,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"
1554,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)"


In [19]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [20]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(29.805547, -98.475534)",29.805547,-98.475534
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
856,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
982,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)",30.211452,-81.617981
1103,BELVEDERE DRIVE,2.340000e+06,"(44.7075622, -63.5455988)",44.707562,-63.545599
1216,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)",51.521201,-0.158908
1254,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1554,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)",51.589961,-0.318525


In [21]:
df = df_affordable.drop(columns=['city_coord'])

In [22]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,29.805547,-98.475534
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
856,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
982,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981
1103,BELVEDERE DRIVE,2.340000e+06,44.707562,-63.545599
1216,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908
1254,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1554,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525


In [23]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [65]:
# import folium 

In [67]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

NameError: name 'folium' is not defined

In [32]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3
CLIENT_SECRET:QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [36]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
5,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
6,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
7,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant
8,ALBION SQUARE,-41.273758,173.289393,Sprig & Fern,-41.274508,173.286527,Brewery
9,ALBION SQUARE,-41.273758,173.289393,Hopgood's,-41.274749,173.283831,Restaurant


In [37]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,28,28,28,28,28,28
ANSDELL TERRACE,45,45,45,45,45,45
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,14,14,14,14,14,14
BEAUCLERC ROAD,6,6,6,6,6,6
BELVEDERE DRIVE,1,1,1,1,1,1
BICKENHALL STREET,64,64,64,64,64,64
BIRCHLANDS AVENUE,10,10,10,10,10,10
BRAMPTON GROVE,3,3,3,3,3,3


In [39]:
 # get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 349 uniques categories.


In [40]:
location_venues.shape

(4523, 7)

In [41]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,ALBION SQUARE,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,ANSDELL TERRACE,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,APPLEGARTH ROAD,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,BARONSMEAD ROAD,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,BEAUCLERC ROAD,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
5,BELVEDERE DRIVE,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
6,BICKENHALL STREET,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.015625,0.0,0.015625,0.015625,0.0,0.0
7,BIRCHLANDS AVENUE,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
8,BRAMPTON GROVE,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
9,BRIARDALE GARDENS,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [43]:
london_grouped.shape

(147, 350)

In [44]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.18
1  Indian Restaurant  0.07
2        Coffee Shop  0.07
3                Pub  0.07
4                Bar  0.07


----ANSDELL TERRACE----
                venue  freq
0           Juice Bar  0.09
1               Hotel  0.07
2          Restaurant  0.07
3   Indian Restaurant  0.07
4  Italian Restaurant  0.04


----APPLEGARTH ROAD----
                   venue  freq
0                 Casino  0.25
1                    Pub  0.25
2              Nightclub  0.25
3         Sandwich Place  0.25
4  Outdoors & Recreation  0.00


----BARONSMEAD ROAD----
               venue  freq
0  Food & Drink Shop  0.14
1     Farmers Market  0.07
2               Park  0.07
3      Movie Theater  0.07
4               Café  0.07


----BEAUCLERC ROAD----
             venue  freq
0       Hobby Shop  0.17
1        Speakeasy  0.17
2              Spa  0.17
3  Harbor / Marina  0.17
4      Pizza Place  0.17


----BELVEDERE DRIVE----
                   ve

In [45]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [47]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [48]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Indian Restaurant,Bar,Coffee Shop,Restaurant,Pub,Fish & Chips Shop,Supermarket,Burger Joint,Museum
1,ANSDELL TERRACE,Juice Bar,Indian Restaurant,Hotel,Restaurant,Pub,Bakery,Clothing Store,Italian Restaurant,Burger Joint,Supermarket
2,APPLEGARTH ROAD,Pub,Nightclub,Sandwich Place,Casino,Flower Shop,Farm,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
3,BARONSMEAD ROAD,Food & Drink Shop,Pub,Thai Restaurant,Pizza Place,Restaurant,Coffee Shop,Park,Farmers Market,Café,Nature Preserve
4,BEAUCLERC ROAD,Speakeasy,Spa,Pizza Place,Hobby Shop,Automotive Shop,Harbor / Marina,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit


In [49]:
venues_sorted.shape

(147, 11)

In [50]:
london_grouped.shape

(147, 350)

In [51]:
london_grouped=df

In [52]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([0, 2, 1, 2, 3, 0, 3, 1, 1, 0, 2, 2, 0, 0, 3, 3, 0, 2, 1, 0, 4, 4,
       2, 0, 0, 1, 2, 4, 0, 1, 2, 3, 2, 3, 3, 4, 2, 2, 3, 1, 0, 3, 4, 1,
       4, 1, 1, 4, 1, 1], dtype=int32)

In [53]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
856,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [54]:
london_grouped_clustering.shape

(161, 4)

In [55]:
df.shape

(161, 4)

In [56]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [57]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [58]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,0,Café,Indian Restaurant,Bar,Coffee Shop,Restaurant,Pub,Fish & Chips Shop,Supermarket,Burger Joint,Museum
390,ANHALT ROAD,2.435000e+06,29.805547,-98.475534,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,1,Juice Bar,Indian Restaurant,Hotel,Restaurant,Pub,Bakery,Clothing Store,Italian Restaurant,Burger Joint,Supermarket
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,2,Pub,Nightclub,Sandwich Place,Casino,Flower Shop,Farm,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
856,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,3,Food & Drink Shop,Pub,Thai Restaurant,Pizza Place,Restaurant,Coffee Shop,Park,Farmers Market,Café,Nature Preserve
982,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,0,Speakeasy,Spa,Pizza Place,Hobby Shop,Automotive Shop,Harbor / Marina,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1103,BELVEDERE DRIVE,2.340000e+06,44.707562,-63.545599,3,Campground,Zoo Exhibit,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1216,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,1,Coffee Shop,Gastropub,Restaurant,Italian Restaurant,Hotel,Bakery,Garden,Greek Restaurant,Pizza Place,Bar
1254,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,1,Pub,Brewery,Coffee Shop,French Restaurant,Bakery,Train Station,Lake,Chinese Restaurant,Pizza Place,Dance Studio
1554,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,0,Food Service,Construction & Landscaping,Home Service,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [59]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'folium' is not defined

In [60]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Indian Restaurant,Bar,Coffee Shop,Restaurant,Pub,Fish & Chips Shop,Supermarket,Burger Joint,Museum
982,2480000.0,Speakeasy,Spa,Pizza Place,Hobby Shop,Automotive Shop,Harbor / Marina,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1554,2456875.0,Food Service,Construction & Landscaping,Home Service,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1915,2445000.0,Grocery Store,Athletics & Sports,Bar,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
1981,2492500.0,Supermarket,English Restaurant,Dry Cleaner,Fast Food Restaurant,Gym,Café,Hardware Store,Coffee Shop,Park,Rental Car Location


In [61]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Juice Bar,Indian Restaurant,Hotel,Restaurant,Pub,Bakery,Clothing Store,Italian Restaurant,Burger Joint,Supermarket
1216,2208500.0,Coffee Shop,Gastropub,Restaurant,Italian Restaurant,Hotel,Bakery,Garden,Greek Restaurant,Pizza Place,Bar
1254,2217000.0,Pub,Brewery,Coffee Shop,French Restaurant,Bakery,Train Station,Lake,Chinese Restaurant,Pizza Place,Dance Studio
2226,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2639,2250000.0,Bookstore,Pharmacy,Hotel,Gift Shop,Outdoor Supply Store,Pizza Place,Bakery,Department Store,Coffee Shop,Grocery Store


In [62]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,2400000.0,Pub,Nightclub,Sandwich Place,Casino,Flower Shop,Farm,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
1633,2397132.0,Breakfast Spot,Sporting Goods Shop,Grocery Store,Coffee Shop,Sushi Restaurant,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
1798,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2159,2425000.0,Clothing Store,Basketball Court,Women's Store,Convenience Store,Electronics Store,American Restaurant,Stationery Store,Gym,Department Store,Chinese Restaurant


In [63]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
856,2375000.0,Food & Drink Shop,Pub,Thai Restaurant,Pizza Place,Restaurant,Coffee Shop,Park,Farmers Market,Café,Nature Preserve
1103,2340000.0,Campground,Zoo Exhibit,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2069,2375000.0,Pub,Park,Indian Restaurant,Yoga Studio,Hotel,Ice Cream Shop,English Restaurant,Clothing Store,Thai Restaurant,Gastropub
2130,2379652.7,Pub,Indian Restaurant,Hotel,Bakery,Ice Cream Shop,Park,Coffee Shop,Hostel,Yoga Studio,Breakfast Spot
2945,2367500.0,Hotel,Pub,Garden,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Bar,Chinese Restaurant,Mediterranean Restaurant


In [64]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2243,2.300000e+06,Farm,Zoo Exhibit,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant
2407,2.286679e+06,Café,Italian Restaurant,Coffee Shop,Pub,Bar,Park,French Restaurant,Portuguese Restaurant,Bookstore,Gourmet Shop
2687,2.287500e+06,Pub,Brewery,Gym / Fitness Center,Gift Shop,Art Museum,Filipino Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
3378,2.298000e+06,Hotel,Zoo Exhibit,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market
4286,2.265000e+06,Gym,Gym / Fitness Center,Trail,American Restaurant,Dry Cleaner,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
